In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# A bit complicated example...
pumpkins:pd.DataFrame = pd.read_csv('../US-pumpkins.csv')

pumpkins = pumpkins[pumpkins['Package'].str.contains('bushel', case=True, regex=True)]

new_columns = ['Package', 'Variety', 'Low Price', 'High Price', 'Date']
pumpkins = pumpkins.drop([c for c in pumpkins.columns if c not in new_columns], axis=1)

price = (pumpkins['Low Price'] + pumpkins['High Price']) / 2

month = pd.DatetimeIndex(pumpkins['Date']).month
day_of_year = pd.to_datetime(pumpkins['Date']).apply(lambda dt: (dt-datetime(dt.year,1,1)).days)

new_pumpkins = pd.DataFrame({'DayOfYear' : day_of_year, 'Variety': pumpkins['Variety'], 'Package': pumpkins['Package'], 'Price': price})

new_pumpkins.loc[new_pumpkins['Package'].str.contains('1 1/9'), 'Price'] = price/1.1
new_pumpkins.loc[new_pumpkins['Package'].str.contains('1/2'), 'Price'] = price*2

pie_pumpkins = new_pumpkins[new_pumpkins['Variety']=='PIE TYPE']
pie_pumpkins.dropna(inplace=True)

X = pie_pumpkins['DayOfYear']
Y = pie_pumpkins['Price']

sumOfX = X.sum()
sumOfY = Y.sum()
sumOfX2 = (X * X).sum()
sumOfXY = (X * Y).sum()
n = len(X)

slope = (n * sumOfXY - sumOfX * sumOfY) / (n * sumOfX2 - sumOfX * sumOfX)
intercept = (sumOfY - slope * sumOfX) / n

y_pred = slope * X + intercept

mse = np.sqrt(mean_squared_error(Y, y_pred))
print(f'Mean error: {mse:3.3} ({mse/np.mean(y_pred)*100:3.3}%)')

plt.plot(X, y_pred, color='blue', linewidth=3)
plt.scatter(X, Y, color='black')
plt.show()